# Backpropagation Complete Example

Adapted from Udacity.

## Task: Imports

In [1]:
import numpy as np
import pandas as pd

## Read and explore the data

In [2]:
admissions=pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

In [3]:
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)
len(data)


400

In [4]:
data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


In [5]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std


In [6]:
# Split off random 10% of the data for testing
sample = np.random.choice(data.index, size=int(len(data)*0.8), replace=False)
data, test_data = data.loc[sample], data.drop(sample)
#print(len(data))


In [7]:
# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']
len(features_test)

80

In [8]:
len(targets_test)

80

## Activation

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

## Hyper-parameters

In [10]:
n_hidden = 3  # number of hidden units
epochs = 1000
learnrate = 0.05

## Initialise

In [11]:
n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

In [12]:
weights_input_hidden

array([[-0.23346402, -0.08974051,  0.6593675 ],
       [-0.18514447,  0.0969839 ,  0.04724568],
       [ 0.85682748, -0.22626293,  0.54654298],
       [-0.23900173,  0.00499819, -0.03398636],
       [ 0.23954521,  0.02599745, -0.77950897],
       [ 0.48109653,  0.50007996,  0.37404025]])

In [13]:
weights_hidden_output

array([-1.14562003,  0.11260254, -0.23762284])

## Epochs

In [14]:
# Iterate over the epochs
for e in range(epochs):
  
    # Initialise the weights
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    
    # Iterate over each training example
    for x, y in zip(features.values, targets):
        
        # *******************
        ## Forward pass ##
        # *******************
        
        hidden_input = np.dot(x,weights_input_hidden)
        hidden_activations = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_activations,weights_hidden_output))

        # *******************
        ## Backward pass ##
        # *******************
        
        # The network error on this particular example
        error = y - output

        # Output error
        output_error = error * output * (1-output)

        # propagate errors to hidden layer
        hidden_error = weights_hidden_output * output_error * hidden_activations * (1-hidden_activations) 

        # Update the change in weights
        del_w_hidden_output += output_error * hidden_activations
        del_w_input_hidden += hidden_error * x[:,None]

    # Actual weight updates. Dividing by n_records to deal with
    # potentially large gradient steps
    weights_input_hidden += (learnrate * del_w_input_hidden) / n_features
    weights_hidden_output += (learnrate * del_w_hidden_output) / n_features

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_activations = sigmoid(np.dot(x, weights_input_hidden))
        
        # Get the network output
        out = sigmoid(np.dot(hidden_activations,
                             weights_hidden_output))
        
        # Compute the loss
        loss = np.mean((out - targets) ** 2)

        # Check the current loss with the previous one
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.21722495232867667
Train loss:  0.22110515050395602   WARNING - Loss Increasing
Train loss:  0.22438202713738714   WARNING - Loss Increasing
Train loss:  0.2278292865421808   WARNING - Loss Increasing
Train loss:  0.23094465782614093   WARNING - Loss Increasing
Train loss:  0.2331653983673942   WARNING - Loss Increasing
Train loss:  0.23465550582883682   WARNING - Loss Increasing
Train loss:  0.23565062346904114   WARNING - Loss Increasing
Train loss:  0.2363181313775491   WARNING - Loss Increasing
Train loss:  0.23676808386657058   WARNING - Loss Increasing


## Calculate accuracy on test data

In [15]:
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
out

array([0.40421174, 0.46727118, 0.35719155, 0.55292575, 0.53333644,
       0.4123639 , 0.12720518, 0.31720205, 0.35588315, 0.1389227 ,
       0.13130408, 0.33587769, 0.17687528, 0.25148716, 0.59183319,
       0.51856127, 0.24322714, 0.27005792, 0.39055674, 0.22873377,
       0.34021111, 0.13349704, 0.49318255, 0.11720866, 0.20478201,
       0.57019238, 0.26529465, 0.28496408, 0.13685809, 0.34562524,
       0.26124425, 0.37392596, 0.19126207, 0.44021832, 0.41860888,
       0.31390432, 0.40582673, 0.20051206, 0.25578747, 0.13188978,
       0.53213117, 0.31211045, 0.64697505, 0.36274341, 0.2835452 ,
       0.20153301, 0.36406206, 0.27338424, 0.15199159, 0.22543688,
       0.37703438, 0.60398515, 0.18920154, 0.21667266, 0.28074239,
       0.35709675, 0.23084866, 0.26040461, 0.39318371, 0.37490659,
       0.25900416, 0.37080982, 0.36445976, 0.27751615, 0.14740057,
       0.17725608, 0.13669814, 0.34537126, 0.4056822 , 0.53162633,
       0.15152266, 0.57276052, 0.45257804, 0.5932857 , 0.37142

## Predict

In [16]:
predictions = out > 0.5
predictions

array([False, False, False,  True,  True, False, False, False, False,
       False, False, False, False, False,  True,  True, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False,  True,
       False,  True, False, False, False, False, False, False])

## Compute accuracy

In [17]:
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.650
